In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np 
import pandas as pd 

from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score as R2

import warnings
warnings.filterwarnings("ignore")

from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

In [2]:
!wget https://stepic.org/media/attachments/course/524/test_luggage_1.csv
!wget https://stepic.org/media/attachments/course/524/test_luggage_2.csv
!wget https://stepic.org/media/attachments/course/524/test_data_passangers.csv
!wget https://stepic.org/media/attachments/course/524/predict_passangers.csv

--2020-12-20 16:06:51--  https://stepic.org/media/attachments/course/524/test_luggage_1.csv
Resolving stepic.org (stepic.org)... 13.69.75.239
Connecting to stepic.org (stepic.org)|13.69.75.239|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1429 (1.4K) [application/octet-stream]
Saving to: ‘test_luggage_1.csv’

test_luggage_1.csv  100%[===================>]   1.40K  --.-KB/s    in 0s      

2020-12-20 16:06:51 (283 MB/s) - ‘test_luggage_1.csv’ saved [1429/1429]

--2020-12-20 16:06:51--  https://stepic.org/media/attachments/course/524/test_luggage_2.csv
Resolving stepic.org (stepic.org)... 13.69.75.239
Connecting to stepic.org (stepic.org)|13.69.75.239|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 737 [application/octet-stream]
Saving to: ‘test_luggage_2.csv’

test_luggage_2.csv  100%[===================>]     737  --.-KB/s    in 0s      

2020-12-20 16:06:51 (63.3 MB/s) - ‘test_luggage_2.csv’ saved [737/737]

--2020-12-20 16:06:51--

In [3]:
data_1 = pd.read_csv('test_luggage_1.csv')
data_2 = pd.read_csv('test_luggage_2.csv')
data_3 = pd.read_csv('test_data_passangers.csv')
test_data = pd.read_csv('predict_passangers.csv')
all_data = pd.concat([data_1,data_2,data_3])

all_data.head()

,is_prohibited,weight,length,width,type
0,No,69,53,17,Suitcase
1,No,79,52,21,Bag
2,No,82,54,20,Suitcase
3,No,81,50,23,Bag
4,No,84,48,19,Suitcase


In [4]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120 entries, 0 to 29
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   is_prohibited  120 non-null    object
 1   weight         120 non-null    int64 
 2   length         120 non-null    int64 
 3   width          120 non-null    int64 
 4   type           120 non-null    object
dtypes: int64(3), object(2)
memory usage: 5.6+ KB


In [5]:
def get_features(data):
  all_data_dum = data.copy()
  all_data_dum['type'] = (all_data_dum['type'] == "Bag").astype(int)
  all_data_dum['is_prohibited'] = (all_data['is_prohibited'] == "Yes").astype(bool)

  x_train=all_data_dum.drop(columns='is_prohibited')
  y_train=all_data_dum['is_prohibited']

  logit_model=sm.Logit(y_train,x_train)
  fit_model=logit_model.fit()

  pvalues = fit_model.pvalues[fit_model.pvalues < 0.05]
  if pvalues.size == 0:
    return "Prediction makes no sense"
  else:
    return pvalues

In [6]:
result = get_features(all_data)
print("Significant predictors: ")
print(result)

Optimization terminated successfully.
         Current function value: 0.635428
         Iterations 5
Significant predictors: 
type    0.000907
dtype: float64


In [8]:
def suspects(train_data,test_data):
  all_data_dum = train_data.copy()
  all_data_dum['type'] = (all_data_dum['type'] == "Bag").astype(int)
  all_data_dum['is_prohibited'] = (all_data['is_prohibited'] == "Yes").astype(bool)

  x_train=all_data_dum.drop(columns='is_prohibited')
  y_train=all_data_dum['is_prohibited']

  logit_model=sm.Logit(y_train,x_train)
  fit_model=logit_model.fit()

  test_data_1=test_data.drop(columns='passangers')
  test_data_1['type'] = (test_data['type'] == "Bag").astype(int)
  test_data_1.head()

  results = fit_model.predict(test_data_1)
  
  return test_data.iloc[np.where(results == results.max())]['passangers']

In [9]:
suss = suspects(all_data,test_data)
print("Suspects:")
print(suss)

Optimization terminated successfully.
         Current function value: 0.635428
         Iterations 5
Suspects:
1    Anatoliy
Name: passangers, dtype: object
